# Getting started with the EB-NeRD

In [1]:
import polars as pl

In [2]:
df_articles = pl.read_parquet("/kaggle/input/recommender-systems-challenge-2024/ebnerd_large/articles.parquet", columns = ["article_id", "topics"])
df_behaviors = pl.read_parquet("/kaggle/input/recommender-systems-challenge-2024/ebnerd_large/train/behaviors.parquet", columns = ["user_id", "article_id", "impression_time"])
df_history = pl.read_parquet("/kaggle/input/recommender-systems-challenge-2024/ebnerd_large/train/history.parquet", columns = ["user_id", "article_id_fixed","impression_time_fixed"])

In [3]:

df_history = df_history.rename({"article_id_fixed": "article_id", "impression_time_fixed": "impression_time"})

"""df_history = df_history.pipe(
    truncate_history,
    column="article_id",
    history_size=30,
    padding_value=0,
    enable_warning=False,
)"""



df_history = df_history.explode("article_id", "impression_time")

In [4]:
new_df = pl.concat(items = [df_behaviors,df_history])

In [5]:
train_ds = df_articles.explode("topics")

In [6]:
new_df = new_df.join(train_ds, on="article_id")

In [7]:
new_df

user_id,article_id,impression_time,topics
u32,i32,datetime[μs],str
18695,9774652,2023-05-21 21:08:35,"""Kendt"""
18695,9774652,2023-05-21 21:08:35,"""Begivenhed"""
18695,9774652,2023-05-21 21:08:35,"""Sport"""
18695,9774652,2023-05-21 21:08:35,"""Sportsbegivenh…"
18813,9754160,2023-05-21 21:27:16,"""Konflikt og kr…"
…,…,…,…
1541617,9750772,2023-05-08 07:37:32,"""Større transpo…"
1541617,9753521,2023-05-08 07:38:01,"""Økonomi"""
1541617,9753521,2023-05-08 07:38:01,"""Politik"""


In [8]:
new_df = new_df.select(["topics", "impression_time"])
new_df

topics,impression_time
str,datetime[μs]
"""Kendt""",2023-05-21 21:08:35
"""Begivenhed""",2023-05-21 21:08:35
"""Sport""",2023-05-21 21:08:35
"""Sportsbegivenh…",2023-05-21 21:08:35
"""Konflikt og kr…",2023-05-21 21:27:16
…,…
"""Større transpo…",2023-05-08 07:37:32
"""Økonomi""",2023-05-08 07:38:01
"""Politik""",2023-05-08 07:38:01


In [9]:
new_df = new_df.with_columns([
    pl.col("impression_time").dt.strftime("%Y-%m-%d").alias("date"),
    pl.col("topics")

])

In [17]:
new_df = new_df.drop("impression_time")

In [20]:
count_num_topics_per_day = new_df.group_by(['date', 'topics']).len()
count_num_topics_per_day

date,topics,len
str,str,u32
"""2023-05-21""","""Sport""",109506
"""2023-05-21""","""Sundhed""",34675
"""2023-05-21""","""Sygdom og beha…",34093
"""2023-05-21""","""Videnskab""",8012
"""2023-05-21""","""Naturvidenskab…",7780
…,…,…
"""2023-05-07""","""Mærkedag""",120
"""2023-05-04""","""Religion""",146
"""2023-04-29""","""Mærkedag""",6


In [38]:
import pandas as pd
df = count_num_topics_per_day.to_pandas()

In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

df['date'] = pd.to_datetime(df['date'])
df['len'] = df['len'].astype(int)


valori_unici = df['topics'].unique()


def plot_interattivo(valore_selezionato):
    plt.figure(figsize=(10, 6))

    df_filtrato = df[df['topics'] == valore_selezionato]
    
    
    df_filtrato = df_filtrato.sort_values(by='date')
    
   
    plt.plot(df_filtrato['date'], df_filtrato['len'], marker='o', linestyle='-')
    
  
    plt.xlabel('Data')
    plt.ylabel('Topics per date')
    #plt.yscale("log")
    
    plt.xticks(df_filtrato["date"]) 
    
    plt.title(f'Plot per il topic: {valore_selezionato}')
    
    plt.xticks(rotation=45)
    
    last_date = df_filtrato['date'].iloc[-7]
    plt.axvline(x=last_date, color='red', linestyle='--')
    
    plt.show()

menu_scelta_nome = widgets.Dropdown(options=valori_unici, description='Seleziona nome:')

widgets.interactive(plot_interattivo, valore_selezionato=menu_scelta_nome)


interactive(children=(Dropdown(description='Seleziona nome:', options=('Sport', 'Sundhed', 'Sygdom og behandli…

In [58]:
df_filtrato = df[df['topics'] == "Udlejning"]
df_filtrato = df_filtrato.sort_values(by='date')
df_filtrato

,date,topics,len
1130,2023-04-27,Udlejning,338
1514,2023-04-28,Udlejning,17469
181,2023-04-29,Udlejning,141114
759,2023-04-30,Udlejning,79859
2147,2023-05-01,Udlejning,3920
2193,2023-05-02,Udlejning,1131
2247,2023-05-03,Udlejning,239
2272,2023-05-04,Udlejning,181
479,2023-05-05,Udlejning,40970
1929,2023-05-06,Udlejning,164247
